## Importing the necessary modules

In [1]:
import re
import os
import sys
import spacy
import random
import numpy as np
import panel as pn
import pandas as pd
import hvplot.pandas
import matplotlib_inline
from pathlib import Path
import plotly.express as px
from datetime import datetime
import plotly.graph_objects as go
import matplotlib.ticker as mticker
from matplotlib import pyplot as plt

# Initializing panel

In [2]:
pn.extension('tabulator')
pn.extension('plotly')

### Read the Mpesa transactions from the json file

In [3]:
RawTransactionsDf = pd.read_json("MPESAsms-2023-02-24_12-49-02.json")

### Format The transactions Accoringly

In [4]:
# Getting the columns to be used in the analysis
ValuableInsightsDf = RawTransactionsDf.get(['_id', 'date', 'text'])
# Format the Date and Time Column
WorkingDf = ValuableInsightsDf.assign(
    Date=[datetime.strftime(Time, "%G-%m-%e") for Time in ValuableInsightsDf['date']], 
    Time=[datetime.strftime(Time, "%H:%M:%S") for Time in ValuableInsightsDf['date']]
)[['Date', 'Time', 'text']]

WorkingDf

,Date,Time,text
0,2020-05-27,12:15:07,"OER2HSGN8K Confirmed. Ksh4,100.00 paid to PHON..."
1,2020-05-27,21:31:24,OES8I7MIOO confirmed.You bought Ksh100.00 of a...
2,2020-05-28,08:19:33,OES3IEVNRZ confirmed.You bought Ksh100.00 of a...
3,2020-05-28,15:05:54,OES6IT9PXM confirmed.You bought Ksh50.00 of ai...
4,2020-05-28,15:37:20,OES3IURXA5 confirmed.You bought Ksh50.00 of ai...
...,...,...,...
2303,2023-02-22,19:32:50,RBM29XWHHO Confirmed. Ksh200.00 paid to Quick ...
2304,2023-02-23,05:55:34,RBN1ABKZXR Confirmed. Ksh50.00 sent to Safaric...
2305,2023-02-23,21:08:35,RBO5CKYVYT Confirmed. Ksh50.00 sent to Safaric...
2306,2023-02-24,07:11:12,RBO0D3A0JK Confirmed. Ksh10.00 sent to Safaric...


### Cleaning the data frame

In [17]:
def clean_data():
    Transactions = list()
    # Defining various patterns to be used
    DepositPtn = re.compile(r'cash\s+to\s+([A-z0-9.,\s]+)\s+New')
    withdrawnPtn = re.compile(r'Withdraw\s+Ksh[0-9]*.[0-9]*|Withdraw\s+Ksh[0-9]*.[0-9]*.[0-9]*')
    SendPtn = re.compile(r'\s+(Ksh\d*.{0,4})?(\d*.\d+)\s+sent|\s+sent\s+(Ksh\d*.{0,4})?(\d*.\d+)\s+to')
    paidPtn = re.compile(r'\s+(Ksh\d*.{0,4})?(\d*.\d+)\s+paid')
    RecvPtn = r'received\s+(Ksh\d*.{0,4})?(\d*\.\d+)'
    AgentDetails = re.compile(r'(\d*)\s+-\s+(\w*\s+.*New)')
    DatePtn = re.compile(r'\d+/\d+/\d+|\d+/\d+/\d+\d+')
    TimePtn = re.compile(r'\d+:\d+\s+AM|\d+:\d+\s+PM')
    airTimePtn = re.compile(r'bought\s+(Ksh\d*.{0,4})?(\d*.\d+)\s+of\s+airtime')
    for transaction in WorkingDf['text']:
        if not re.search('Failed', transaction, re.IGNORECASE):
            if re.findall(withdrawnPtn, transaction):
                Amount = re.findall(withdrawnPtn, transaction)[0].removeprefix('Withdraw').strip()
                Agent = (re.findall(AgentDetails, transaction)[0])[1].removesuffix(' New').strip()
                TimeStamp = datetime.strftime(datetime.strptime(re.findall(TimePtn, transaction)[0], '%I:%M %p'), '%I:%M %p')
                DateStamp = datetime.strftime(datetime.strptime(re.findall(DatePtn, transaction)[0], '%d/%m/%y'), '%d/%m/%y')
                Type = 'WITHDRAW'
                CleanAmount = round(float(Amount.removeprefix('Ksh').replace(',', '')),2)
                Transactions.append({'Date':DateStamp, 'Time':TimeStamp, 'Amount':CleanAmount, 'Type': Type, 'Address':Agent})
            elif re.findall(DepositPtn, transaction):
                Amount = re.findall(re.compile(r'Give\s+Ksh[0-9]*.[0-9]*.[0-9]*'), transaction)[0].removeprefix('Give Ksh').strip()
                Agent = re.findall(DepositPtn, transaction)[0].strip()
                TimeStamp = datetime.strftime(datetime.strptime(re.findall(TimePtn, transaction)[0], '%I:%M %p'), '%I:%M %p')
                DateStamp = datetime.strftime(datetime.strptime(re.findall(DatePtn, transaction)[0], '%d/%m/%y'), '%d/%m/%y')
                Type = 'DEPOSIT'
                Transactions.append({'Date':DateStamp, 'Time':TimeStamp, 'Amount':round(float(Amount.replace(',', '')),2), 'Type': Type, 'Address':Agent.strip()})
            elif re.findall(RecvPtn, transaction):
                Amnt = re.findall(RecvPtn, transaction)
                mnt = str()
                for i in Amnt[0]:
                    mnt += i
                Amount = mnt
                Type = 'RECEIVED'
                sender = re.findall(r'from\s+([A-z]*\s+[A-z]*)\s+[0-9]*|from\s+([A-z]*\d*)', transaction)[0]
                for name in sender:
                    if re.findall(r'[A-z]', name):
                        sender = name
                if 'Post Office' in sender:
                    sender = 'POSTBANK BULK'
                TimeStamp = datetime.strftime(datetime.strptime(re.findall(TimePtn, transaction)[0], '%I:%M %p'), '%I:%M %p')
                try:
                    DateStamp = datetime.strftime(datetime.strptime(re.findall(DatePtn, transaction)[0], '%d/%m/%y'), '%d/%m/%y')
                except ValueError:
                    DateStamp = datetime.strftime(datetime.strptime(re.findall(DatePtn, transaction)[0], '%d/%m/%Y'), '%d/%m/%y')
                Transactions.append({'Date':DateStamp, 'Time':TimeStamp, 'Amount':round(float(Amount.removeprefix('Ksh').replace(',', '')),2), 'Type': Type, 'Address':sender.strip()})
            elif re.findall(paidPtn, transaction):
                Amnt = re.findall(paidPtn, transaction)
                mnt = str()
                for i in Amnt[0]:
                    mnt += i
                Amount = mnt
                TimeStamp = datetime.strftime(datetime.strptime(re.findall(TimePtn, transaction)[0], '%I:%M %p'), '%I:%M %p')
                try:
                    DateStamp = datetime.strftime(datetime.strptime(re.findall(DatePtn, transaction)[0], '%d/%m/%y'), '%d/%m/%y')
                except ValueError:
                    DateStamp = datetime.strftime(datetime.strptime(re.findall(DatePtn, transaction)[0], '%d/%m/%Y'), '%d/%m/%y')
                Type = 'PAYMENT'
                try:
                    recipients = re.findall(r'paid\s+to\s+(\w*\s+\w*\s+\d*)\.\s+on|paid\s+to\s+(\w*\s+\w*\s+\w*\d+)\.\s+on|paid\s+to\s+(\w*\s+\w*)\.\s+on|paid\s+to\s+(\w*.*\w*\s+\w*)\.\s+on|paid\s+to\s+(\w+.\w*)\.\s+on|paid\s+to\s+(\w*\s+\w*\s+\w*.\w*)\.\s+on |paid\s+to\s+(\w*\s+\w*.*)\.\s+on', transaction)[0]
                except IndexError:
                    print(transaction)
                    sys.exit(1)
                for rcp in recipients:
                    if re.findall(r'[A-z]', rcp):
                        Transactions.append({'Date':DateStamp, 'Time':TimeStamp, 'Amount':round(float(Amount.removeprefix('Ksh').replace(',', '')),2), 'Type': Type, 'Address':rcp.strip()})
            elif re.findall(airTimePtn, transaction):
                Amnt = re.findall(airTimePtn, transaction)
                mnt = str()
                for i in Amnt[0]:
                    mnt += i
                Amount = mnt
                TimeStamp = datetime.strftime(datetime.strptime(re.findall(TimePtn, transaction)[0], '%I:%M %p'), '%I:%M %p')
                try:
                    DateStamp = datetime.strftime(datetime.strptime(re.findall(DatePtn, transaction)[0], '%d/%m/%y'), '%d/%m/%y')
                except ValueError:
                    DateStamp = datetime.strftime(datetime.strptime(re.findall(DatePtn, transaction)[0], '%d/%m/%Y'), '%d/%m/%y')
                Type = 'PAYMENT'
                Transactions.append({'Date':DateStamp, 'Time':TimeStamp, 'Amount':round(float(Amount.removeprefix('Ksh').replace(',', '')),2), 'Type': Type, 'Address':'Safaricom'})
            elif re.findall(SendPtn, transaction):
                Amnt = re.findall(SendPtn, transaction)
                mnt = str()
                for i in Amnt[0]:
                    mnt += i
                Amount = mnt
                TimeStamp = datetime.strftime(datetime.strptime(re.findall(TimePtn, transaction)[0], '%I:%M %p'), '%I:%M %p')
                try:
                    DateStamp = datetime.strftime(datetime.strptime(re.findall(DatePtn, transaction)[0], '%d/%m/%y'), '%d/%m/%y')
                except ValueError:
                    DateStamp = datetime.strftime(datetime.strptime(re.findall(DatePtn, transaction)[0], '%d/%m/%Y'), '%d/%m/%y')
                Type = 'SENT'
                sender = re.findall(r'\sto\s(.*?)\s(on|for)\b', transaction, re.I)
                if sender:
                    sender = sender[0][0].strip()
                    if re.findall(r'[0-9]$', sender):
                        sender =  sender.removesuffix(re.findall(r'\s+[0-9]*$', sender)[0])
                    if sender.startswith(('Safaricom', 'SAFARICOM')):
                        Type = 'PAYMENT'
                        sender = 'Safaricom'
                    if 'TELKOM' in sender:
                        Type = 'PAYMENT'
                    Transactions.append({'Date':DateStamp, 'Time':TimeStamp, 'Amount':round(float(Amount.removeprefix('Ksh').replace(',', '')),2), 'Type': Type, 'Address':sender})
                else:
                    print(transaction)
            else:
                pass
    # Cleaned Dataframe
    Maindf = pd.DataFrame(Transactions)
    Maindf.to_csv('FullTransactions.csv', index=False)
    return Maindf

In [6]:
clean_data()

,Date,Time,Amount,Type,Address
0,27/05/20,03:15 PM,4100.0,PAYMENT,PHONELINK JUNCTION 1
1,28/05/20,12:31 AM,100.0,PAYMENT,Safaricom
2,28/05/20,11:19 AM,100.0,PAYMENT,Safaricom
3,28/05/20,06:05 PM,50.0,PAYMENT,Safaricom
4,28/05/20,06:35 PM,50.0,PAYMENT,Safaricom
...,...,...,...,...,...
2184,22/02/23,10:32 PM,200.0,PAYMENT,Quick Mart Rongai Express
2185,23/02/23,08:55 AM,50.0,PAYMENT,Safaricom
2186,24/02/23,12:08 AM,50.0,PAYMENT,Safaricom
2187,24/02/23,10:11 AM,10.0,PAYMENT,Safaricom


### Get the Total Amount Transacted per Address

In [7]:
def transactionPerAddr():
    Sum_Expenditure = clean_data()[['Address', 'Type', 'Amount']].assign(Transactions=1)
    Sum_Expenditure = Sum_Expenditure.assign(
        SENT=Sum_Expenditure.Amount.where(Sum_Expenditure.Type == 'SENT').fillna(0.0), 
        RECEIVED=Sum_Expenditure.Amount.where(Sum_Expenditure.Type == 'RECEIVED').fillna(0.0),
        PAYMENT=Sum_Expenditure.Amount.where(Sum_Expenditure.Type == 'PAYMENT').fillna(0.0), 
        DEPOSIT=Sum_Expenditure.Amount.where(Sum_Expenditure.Type == 'DEPOSIT').fillna(0.0),
        WITHDRAW=Sum_Expenditure.Amount.where(Sum_Expenditure.Type == 'WITHDRAW').fillna(0.0)
    )
    Sum_Expenditure = Sum_Expenditure[['Address', 'SENT', 'RECEIVED', 'DEPOSIT', 'PAYMENT', 'WITHDRAW', 'Amount', 'Transactions']].groupby(
        'Address', as_index=False
    ).sum()
    Sum_Expenditure = Sum_Expenditure.rename(columns={'Amount':'Total Amount'})
    Sum_Expenditure.to_csv('Expenditure.csv', index=False)
    return Sum_Expenditure

In [8]:
transactionPerAddr()

,Address,SENT,RECEIVED,DEPOSIT,PAYMENT,WITHDRAW,Total Amount,Transactions
0,ALEX MULINGE,0.0,20.0,0.0,0.0,0.0,20.0,1
1,ALICE WAITHERA NDUNGU,0.0,0.0,0.0,220.0,0.0,220.0,3
2,ALLAN WEBO,300.0,100.0,0.0,0.0,0.0,400.0,7
3,ALVIK STOP OVER RESORT,0.0,0.0,0.0,120.0,0.0,120.0,1
4,AMOS OMWAKA,300.0,0.0,0.0,0.0,0.0,300.0,2
...,...,...,...,...,...,...,...,...
301,paul kithuka,100.0,0.0,0.0,0.0,0.0,100.0,1
302,ryan okal,200.0,0.0,0.0,0.0,0.0,200.0,1
303,stephen ageroni,30.0,0.0,0.0,0.0,0.0,30.0,1
304,winfred musyoka,30.0,0.0,0.0,0.0,0.0,30.0,1


### A function to Get The Cash Flow

In [9]:
def cashflowInspection(flowType):
    workFrame = transactionPerAddr()
    majorInflow = workFrame.sort_values(by=['RECEIVED', 'Transactions'], ascending=False).reset_index()[['Address', 'RECEIVED', 'Transactions']].head(5).rename(columns={'RECEIVED':'Amount'})
    majorOutflow = workFrame.sort_values(by=['SENT', 'Transactions'], ascending=False).reset_index()[['Address', 'SENT', 'Transactions']].head(5).rename(columns={'SENT':'Amount'})
    highBills = workFrame.sort_values(by=['PAYMENT', 'Transactions'], ascending=False).reset_index()[['Address', 'PAYMENT', 'Transactions']].head(5).rename(columns={'PAYMENT':'Amount'})
    frequentie = workFrame.sort_values(by=['Transactions', 'Total Amount'], ascending=False).reset_index()[['Address', 'Total Amount', 'Transactions']].head(5).rename(columns={'Total Amount':'Amount'})
    if flowType == 'Major Cash Inflows':
        return majorInflow
    elif flowType == 'Major Cash Outflows':
        return majorOutflow
    elif flowType == 'Most Frequent Transactions':
        return frequentie
    elif flowType == 'High Bills':
        return highBills
    else:
        return majorInflow

### Prepare data to used for monthly analysis

In [10]:
def monthlyAnalysisData():
    Maindf = clean_data()
    Date_Activity = Maindf[['Date', 'Type', 'Amount']]
    MONTHS, MONTHS_Names, Exp, YRS = [], [], [], [datetime.strftime(datetime.strptime(yr, '%d/%m/%y'), '%m-%Y') for yr in Date_Activity['Date']]

    # Get the years that have been featured in the transactions
    UYRS = list(set([datetime.strftime(datetime.strptime(yr, '%m-%Y'), '%Y') for yr in YRS]))
    Big  = dict(zip(UYRS, np.arange(len(UYRS))))

    for YKINDX, YKEY in enumerate(Big):
        Mnames = {'Jan':[], 'Feb':[], 'Mar':[], 'Apr':[], 'May':[], 'Jun':[], 'Jul':[], 'Aug':[], 'Sep':[], 'Oct':[], 'Nov':[], 'Dec':[]}
        for _, MKEY in enumerate(Mnames):
            MDETAILS = list()
            for indx, det in enumerate(YRS):
                if YKEY == datetime.strftime(datetime.strptime(det, '%m-%Y'), '%Y') and MKEY == datetime.strftime(datetime.strptime(det, '%m-%Y'), '%b'):
                    MDETAILS.append(Date_Activity.values[indx])

            Mnames[MKEY] = MDETAILS

        acc = {'Month':[],'Transactions':[],'RECEIVED':[],'DEPOSIT':[],'SENT':[], 'WITHDRAW':[],'PAYMENTS':[],'TotalSpent(Ksh.)':[]}
        full = list()
        for MDETAIL in Mnames.keys():
            Total_Exp, Total_Dep, Total_With = 0, 0, 0
            Total_Sent = 0
            Total_Recv = 0
            Total_Paid = 0
            for m in Mnames.get(MDETAIL):
                # Calculate the total amount spent
                if m[1] == 'SENT':
                    Total_Sent = Total_Sent + m[2]
                    Total_Exp = Total_Exp + m[2]
                elif m[1] == 'RECEIVED':
                    Total_Recv = Total_Recv + m[2]
                    Total_Exp = Total_Exp
                elif m[1] == 'PAYMENT':
                    Total_Paid = Total_Paid + m[2]
                    Total_Exp = Total_Exp + m[2]
                elif m[1] == 'DEPOSIT':
                    Total_Dep = Total_Dep + m[2]
                    Total_Exp = Total_Exp + m[2]
                elif m[1] == 'WITHDRAW':
                    Total_With = Total_With + m[2]
                    Total_Exp = Total_Exp + m[2]
                else:
                    pass
            acc['Month'].append(MDETAIL)
            acc['SENT'].append(Total_Sent)
            acc['RECEIVED'].append(Total_Recv)
            acc['PAYMENTS'].append(Total_Paid)
            acc['DEPOSIT'].append(Total_Dep)
            acc['WITHDRAW'].append(Total_With)
            acc['Transactions'].append(len(Mnames.get(MDETAIL)))
            acc['TotalSpent(Ksh.)'].append(Total_Exp)
            full.append(acc)
        Big[YKEY] = acc
        pd.DataFrame(Big[YKEY]).to_csv(f'{YKEY}Expenditure.csv', mode='w', index=False)

### Read The Data prepared in the previous step and visualize them

In [11]:
def GoMonthly():
    Frame22 = pd.read_csv('2022Expenditure.csv').iloc[:,1:].sum().to_frame(name='2022').reset_index().rename(columns={'index':'Type'})
    Frame21 = pd.read_csv('2021Expenditure.csv').iloc[:,1:].sum().to_frame(name='2021').reset_index().rename(columns={'index':'Type'})
    Frame20 = pd.read_csv('2020Expenditure.csv').iloc[:,1:].sum().to_frame(name='2020').reset_index().rename(columns={'index':'Type'})
    Frame23 = pd.read_csv('2023Expenditure.csv').iloc[:,1:].sum().to_frame(name='2023').reset_index().rename(columns={'index':'Type'})
    combined_df = pd.merge(Frame20, Frame21, on='Type', how='outer')
    combined_df = pd.merge(combined_df, Frame22, on='Type', how='outer')
    combined_df = pd.merge(combined_df, Frame23, on='Type', how='outer')
    stackedChart = px.bar(combined_df, combined_df.columns, combined_df['Type'], title="Transaction Visualization for 2020-2023", labels={"value":"Amount", "variable":"Years"}, barmode='stack')
    stackedChart.update_layout(
        xaxis_title="Amount",
        plot_bgcolor='rgba(0,0,0,0)',  # set the background color to fully transparent
        paper_bgcolor='rgba(0,0,0,0)'  # set the paper color to fully transparent
    )
    FrameDisp = combined_df
    FrameDisp['Total'] = combined_df.iloc[:,1:].sum(axis=1)
    return pn.pane.Plotly(stackedChart), FrameDisp

### Load The different Data for various years involved in the analysis

In [12]:
def YearlyPie(year):
    # Create a line graph visualizing the expenditures of various years
    if year == '2020':
        MFrame = pd.read_csv('2020Expenditure.csv')
    elif year == '2021':
        MFrame = pd.read_csv('2021Expenditure.csv')
    elif year == '2022':
        MFrame = pd.read_csv('2022Expenditure.csv')
    else:
        MFrame = pd.read_csv('2023Expenditure.csv')
        
    return MFrame

### Visualize the total annual expenditure for the years invlove

In [13]:
def GeneralAnnualExp():
    """
        Get the dataframe and select the bottom row on the dataframe which is Total Amount SPent in the yaer.
        Reshape the dataframe using the T
        
    """

    TotalSpent_data = GoMonthly()[1].set_index('Type').iloc[4,0:4].T.reset_index().rename(columns={'index':'Years'})
    Received_data = GoMonthly()[1].set_index('Type').iloc[2,0:4].T.reset_index().rename(columns={'index':'Years'})

    fig, ax = plt.subplots(figsize=(5.5, 4.5))
    ax.set_title('Annual Expenditure Vs Income', loc='left',fontdict={'color':'grey'})
    ax.bar(x=np.arange(TotalSpent_data['Years'].count())-.23, height=TotalSpent_data['TotalSpent(Ksh.)'], width=.40)
    ax.bar(x=Received_data['Years'], height=Received_data['RECEIVED'], width=.40, color='green', align='edge')
    ax.grid(axis='y', color='black', linestyle='-.', alpha=.1)
    ax.set_xlabel('Years', fontdict={'color':'grey','weight':'bold'})
    ax.set_xticks(np.arange(Received_data['Years'].count()))
    ax.set_xticklabels(Received_data['Years'])
    ax.legend({'Spent':TotalSpent_data['TotalSpent(Ksh.)'], 'Received':Received_data['RECEIVED']})
    ax.spines[['top', 'right', 'left']].set_color(None)
    plt.savefig('AnnualExpVsInc.png', transparent=True)
    plt.close()
    return pn.pane.PNG('AnnualExpVsInc.png', aspect_ratio='auto')

### Categorize the expenditures and visualize them

In [14]:
def ExpenditureCategories(year):
    Categorized_df = clean_data()
    Categorized_df['Category'] = Categorized_df.apply(
        lambda row: 'airtime' if row['Address'].lower() in ['safaricom', 'telkom kenya limited'] and row['Type'].upper() in ['SENT', 'PAYMENT'] 
        else 'School Fees' if row['Address'].lower() == 'multimedia university of kenya via kcb' and row['Type'].upper() in ['SENT', 'PAYMENT'] 
        else 'Shopping' if row['Address'].lower() in [
            'naivas rongai', 'basil com co ltd winner shop rongai rongai', 'vision one stop shop',
            'basil comms sam electronics rongai', 'quick mart rongai', 'quick mart rongai express'
        ] and row['Type'].upper() in ['SENT', 'PAYMENT'] 
        else 'Food' if row['Address'].lower() in [
            'baraka rescue hotel via coop bank', 'dan allan okello', 'rubi restaurant limited1',
            'harmony gen shop', 'kamoke maize mill', 'rebecca mburu', 'richard odongo', 'sailors delight', 
            'serah nthambi', 'krunchies cafa%'
        ] and row['Type'].upper() in ['SENT', 'PAYMENT'] 
        else 'Electricity' if row['Address'].lower() == 'kplc prepaid' and row['Type'].upper() in ['SENT', 'PAYMENT'] 
        else 'Rent' if row['Address'].lower() == 'co-operative bank collection account' and row['Type'].upper() in ['SENT', 'PAYMENT'] 
        else 'Miscellaneous' if row['Type'].upper() in ['SENT', 'PAYMENT'] else 'others',
        axis=1
    )
    Annual_Categorizer = Categorized_df[['Category', 'Amount']].where(Categorized_df.Date.apply(
        lambda dt: datetime.strftime(datetime.strptime(dt, '%d/%m/%y'), '%Y')) == year).dropna().reset_index(drop=True)
    Annual_Categorizer = Annual_Categorizer.groupby('Category', as_index=False).sum()[:-1]
        
    # Draft a data frame for the monthly analysis
    Monthly_Categorizer = Categorized_df[['Date','Category', 'Amount']].where(Categorized_df.Date.apply(
        lambda dt: datetime.strftime(datetime.strptime(dt, '%d/%m/%y'), '%Y')) == year).dropna()
    Monthly_Categorizer['Months'] = Monthly_Categorizer.Date.apply(
        lambda dt: 
        'Jan' if datetime.strftime(datetime.strptime(dt, '%d/%m/%y'), '%m') == '01'
        else 'Feb' if datetime.strftime(datetime.strptime(dt, '%d/%m/%y'), '%m') == '02'
        else 'Mar' if datetime.strftime(datetime.strptime(dt, '%d/%m/%y'), '%m') == '03'
        else 'Apr' if datetime.strftime(datetime.strptime(dt, '%d/%m/%y'), '%m') == '04'
        else 'May' if datetime.strftime(datetime.strptime(dt, '%d/%m/%y'), '%m') == '05'
        else 'Jun' if datetime.strftime(datetime.strptime(dt, '%d/%m/%y'), '%m') == '06'
        else 'Jul' if datetime.strftime(datetime.strptime(dt, '%d/%m/%y'), '%m') == '07'
        else 'Aug' if datetime.strftime(datetime.strptime(dt, '%d/%m/%y'), '%m') == '08'
        else 'Sep' if datetime.strftime(datetime.strptime(dt, '%d/%m/%y'), '%m') == '09'
        else 'Oct' if datetime.strftime(datetime.strptime(dt, '%d/%m/%y'), '%m') == '10'
        else 'Nov' if datetime.strftime(datetime.strptime(dt, '%d/%m/%y'), '%m') == '11'
        else 'Dec' if datetime.strftime(datetime.strptime(dt, '%d/%m/%y'), '%m') == '12'
        else 'Unidentified'
    )

    Monthly_df = Monthly_Categorizer.iloc[:,1:].groupby(['Months', 'Category'], as_index=False).sum()
    Monthly_df = Monthly_df.where(Monthly_df.Category!='others').dropna()
    return Annual_Categorizer, Monthly_df

### The Main Function that calls the other functions and binds them together

In [15]:
def main():
    MonthlyFrame, MonthlyStackChart = GoMonthly()[1], GoMonthly()[0]
    TotalSpent = round(MonthlyFrame.iloc[4,5], 2)
    TotalReceived = round(MonthlyFrame.iloc[2,5], 2)
    TotalTransactions = int(MonthlyFrame['Total'].to_list()[0])
    AnnualDfi = YearlyPie('2020')
    cashflowInspection('Major Cash Outflows')

    CashFlowBtn = pn.widgets.Select(
        options=['Major Cash Outflows', 'Major Cash Inflows', 'High Bills', 'Most Frequent Transactions']
    )

    transactionType = pn.widgets.Select(
        options=['Financial Outflow','Financial Inflow']
    )

    YearsButton = pn.widgets.RadioButtonGroup(
        name='Y-axis', options=['2020','2021','2022','2023'], 
        button_type='warning'
    )

    
    # Initial AnnualDfi Frame
    icashFlow = cashflowInspection('Major Cash Outflows')
    icashFlow_widget = pn.widgets.Tabulator(icashFlow, aspect_ratio='auto', show_index=False)
    AnnulStats = AnnualDfi.iloc[:,1:].sum()
        
    AnnulInfl_widget = pn.widgets.Number(
        value=AnnulStats.iloc[1], 
        name="<span style='border-left: 5px green solid; padding: 5px;'>Total Inflow</span>", 
        default_color="green", font_size='20pt', 
        format='<span style="border-left: 5px green solid; padding: 5px;">Ksh. {value:,}</span>')
    AnnulOutfl_widget = pn.widgets.Number(
        value=AnnulStats.iloc[6], 
        name="<span style='border-left: 5px gold solid; padding: 5px;'>Total Outflow</span>", 
        default_color="gold", font_size='20pt', 
        format='<span style="border-left: 5px gold solid; padding: 5px;">Ksh. {value:,}<span>')
    AnnulTran_widget = pn.widgets.Number(
        value=int(AnnulStats.iloc[0]), 
        name="<span style='border-left: 5px grey solid; padding: 5px;'>Total Transactions</span>", 
        default_color="grey", font_size='20pt', 
        format='<span style="border-left: 5px grey solid; padding: 5px;">{value:,}</span>')
            
    TotalSpentDf = AnnualDfi[['Month','SENT', 'PAYMENTS', 'WITHDRAW']]
    shepu = px.bar(TotalSpentDf, y=TotalSpentDf.columns[1:], barmode='group', x='Month')

    # Update layout for better visualization
    shepu.update_layout(
        height=450,
        title='2020 Financial Outflow Trend Analysis',
        xaxis_title='Month',
        yaxis_title='Amount',
        title_font_color='green',
        plot_bgcolor='rgba(0,0,0,0)',  # set the background color to fully transparent
        paper_bgcolor='rgba(0,0,0,0)'  # set the paper color to fully transparent
        # legend=dict(x=0, y=1, traceorder='normal'),
    )
    AnDfBar_widget = pn.pane.Plotly(shepu)
    
    # Call the categorical expenditures
    Categorized_Exp_Table, Monthly_Exp_Table = ExpenditureCategories('2020')
    
    # Visualize the categories using a pie chart
    Cat_fig = px.pie(Categorized_Exp_Table.Amount, names=Categorized_Exp_Table.Category, hole=.5, hover_data=['Amount'], title="2020 Categorical Expenditure")
    # Format Amount with commas in the hover text
    Cat_fig.update_traces(hovertemplate='%{label}: Ksh. %{value:,.2f}', values=Categorized_Exp_Table.Amount)
    Cat_fig.update_layout(
        width=500,
        title_font_color='green',
        plot_bgcolor='rgba(0,0,0,0)',  # set the background color to fully transparent
        paper_bgcolor='rgba(0,0,0,0)'  # set the paper color to fully transparent
    )
    Cat_fig_widget = pn.pane.Plotly(Cat_fig, aspect_ratio='auto')

    # Visual the monthly expenditure
    Monthly_Group_Bar = px.bar(
        barmode='group', data_frame=Monthly_Exp_Table, y=Monthly_Exp_Table.Amount, 
        x=Monthly_Exp_Table.Months, color=Monthly_Exp_Table.Category, 
        category_orders={'Months': ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']},
        title='Amount Spent in 2020 on Each Category Across The Months'
    )
    Monthly_Group_Bar.update_layout(
        width=1150, 
        height=400,
        title_font_color='green',
        plot_bgcolor='rgba(0,0,0,0)',  # set the background color to fully transparent
        paper_bgcolor='rgba(0,0,0,0)'  # set the paper color to fully transparent
    )
    Monthly_fig_widget = pn.pane.Plotly(Monthly_Group_Bar, aspect_ratio='auto')

    def on_widget_change(event):
        chosenType = CashFlowBtn.value
        icashFlow_widget.value = cashflowInspection(chosenType)

    def year_widget_change(event):
        year = YearsButton.value
        # transactionType.set_option = "Financial Outflow"
        AnnualDfi = YearlyPie(year)
        Categorized_Exp_Table, Monthly_Exp_Table = ExpenditureCategories(year)
        AnnulStats = AnnualDfi.iloc[:,1:].sum()
        AnnulInfl_widget.value = AnnulStats.iloc[1]
        AnnulOutfl_widget.value = AnnulStats.iloc[6]
        AnnulTran_widget.value = int(AnnulStats.iloc[0])
            
        TotalSpentDf = AnnualDfi[['Month','SENT', 'PAYMENTS', 'WITHDRAW']]
        shepu = px.bar(TotalSpentDf, y=TotalSpentDf.columns[1:], barmode='group', x='Month')

        # Update layout for better visualization
        shepu.update_layout(
            height=450,
            title=f'{year} Financial Outflow Trend Analysis',
            xaxis_title='Month',
            yaxis_title='Amount',
            title_font_color='green',
            plot_bgcolor='rgba(0,0,0,0)',  # set the background color to fully transparent
            paper_bgcolor='rgba(0,0,0,0)'  # set the paper color to fully transparent
            # legend=dict(x=0, y=1, traceorder='normal'),
        )
        AnDfBar_widget.object = shepu
            
        Cat_fig = px.pie(
            Categorized_Exp_Table.Amount, names=Categorized_Exp_Table.Category, 
            hole=.5, hover_data=['Amount'], title=f"{year} Categorical Expenditure"
        )
        # Format Amount with commas in the hover text
        Cat_fig.update_traces(hovertemplate='%{label}: Ksh. %{value:,.2f}', values=Categorized_Exp_Table.Amount)
        Cat_fig.update_layout(
            width=500, 
            title_font_color='green',
            plot_bgcolor='rgba(0,0,0,0)',  # set the background color to fully transparent
            paper_bgcolor='rgba(0,0,0,0)'  # set the paper color to fully transparent
        )
        Cat_fig_widget.object = Cat_fig
        
        # Visualize the monthly expenditure
        Monthly_Group_Bar = px.bar(
            barmode='group', data_frame=Monthly_Exp_Table, y=Monthly_Exp_Table.Amount, 
            x=Monthly_Exp_Table.Months, color=Monthly_Exp_Table.Category, 
            category_orders={'Months': ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']}, 
            title=f'Amount Spent in {year} on Each Category Across The Months'
        )
        Monthly_Group_Bar.update_layout(
            width=1150, 
            height=400,
            title_font_color='green',
            # title_align='center',
            plot_bgcolor='rgba(0,0,0,0)',  # set the background color to fully transparent
            paper_bgcolor='rgba(0,0,0,0)'  # set the paper color to fully transparent
        )
        Monthly_fig_widget.object = Monthly_Group_Bar

    def trans_widget_change(event):
        YaxisTransType = transactionType.value
        if YearsButton.value:
            year = YearsButton.value
        else:
            year = '2020'
        AnnualDfi = YearlyPie(year)
        if YaxisTransType == 'Financial Inflow':
            InterestColumns = ['Month','RECEIVED', 'DEPOSIT']
        else:
            InterestColumns = ['Month','SENT', 'PAYMENTS', 'WITHDRAW']
            
        TotalSpentDf = AnnualDfi[InterestColumns]
        shepu = px.bar(TotalSpentDf, y=TotalSpentDf.columns[1:], barmode='group', x='Month')

        # Update layout for better visualization
        shepu.update_layout(
            height=450,
            title=f'{year} {YaxisTransType} Trend Analysis',
            xaxis_title='Month',
            yaxis_title='Amount',
            title_font_color='green',
            plot_bgcolor='rgba(0,0,0,0)',  # set the background color to fully transparent
            paper_bgcolor='rgba(0,0,0,0)'  # set the paper color to fully transparent
            # legend=dict(x=0, y=1, traceorder='normal'),
        )
        AnDfBar_widget.object = shepu            
        
    transactionType.param.watch(trans_widget_change, 'value')
    YearsButton.param.watch(year_widget_change, 'value')
    CashFlowBtn.param.watch(on_widget_change, 'value')

    tabs = pn.Tabs(
        # The First Tab
        (
            'General Stats',
            pn.Column(
                pn.Row(
                    pn.Column(
                        pn.pane.Markdown('<h2 style="color: green">GENERAL TRANSACTION STATISTICS</h2>'), 
                        pn.pane.Markdown('<h4 style="color: green">Transaction Summary</h4>'), 
                        pn.widgets.Tabulator(MonthlyFrame, show_index=False, aspect_ratio='auto'),
                        GeneralAnnualExp(),
                    ), 
                    pn.Column(
                        pn.Row(
                            pn.Column(
                                pn.Row(
                                    pn.indicators.Number(value=TotalReceived, name="<span style='border-left: 5px green solid; padding: 5px;'>Total Received</span>", format='<span style="border-left: 5px green solid; padding: 5px;">Ksh. {value:,}</span>', font_size='20pt', default_color="green"), 
                                    pn.indicators.Number(value=TotalSpent, name="<span style='border-left: 5px red solid; padding: 5px;'>Total Spent</span>", format='<span style="border-left: 5px red solid; padding: 5px;">Ksh. {value:,}</span>', font_size='20pt', default_color="red"), 
                                    pn.indicators.Number(value=TotalTransactions, name="<span style='border-left: 5px grey solid; padding: 5px;'>Total Transactions</span>", format='<span style="border-left: 5px grey solid; padding: 5px;">{value:,}</span>', font_size='20pt', default_color="grey")
                                )
                            ), align='end'
                        ),
                        MonthlyStackChart,
                        pn.pane.Markdown('<h4 style="color: green">Cashflow Summary</h4>'),
                        CashFlowBtn, 
                        icashFlow_widget, 
                    )
                )
            )
        ), 
        # Second Tab
        (
            'Narrowed Stats', 
            pn.Column(
                pn.Row(
                    pn.Column(
                        pn.Row(
                            pn.Column(
                                pn.pane.Markdown('<h2 style="color: green">TARGETED TRANSACTION ANALYSIS</h2>'),
                                transactionType
                            ), 
                            YearsButton, AnnulInfl_widget, AnnulOutfl_widget, AnnulTran_widget
                        ),
                        pn.Row(
                            pn.Column(
                                AnDfBar_widget
                            ),
                            pn.Column(
                                Cat_fig_widget
                            )
                        )
                    )
                ),
                pn.Row(Monthly_fig_widget, align='center'),
            )
        )
    )

    template = pn.template.FastListTemplate(
        title="MPESA TRANSACTION ANALYSIS DASHBOARD",
        main = [pn.Row(pn.Column(pn.Row(tabs)))],
        header_background="#32CD32"
    )

    # template.show(title="MPESA TRANSACTIONS DASHBOARD", open=False, port=35133)
    template.servable()
    template.close_modal()

### Run the Main cell then go to the commandline and run the following command with the servable template option above:
> panel server MpesaDashboard.ipynb

In [18]:
main()